In [1]:
import sqlite3

# Create the SQLite database
conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

# Create the sales table
cursor.execute("""
CREATE TABLE IF NOT EXISTS sales (
    id INTEGER PRIMARY KEY,
    product TEXT,
    region TEXT,
    sales INTEGER
)
""")

# Insert sample data
sample_data = [
    ("Product X", "Region A", 100),
    ("Product X", "Region B", 200),
    ("Product Y", "Region A", 150),
    ("Product Y", "Region B", 300)
]
cursor.executemany("INSERT INTO sales (product, region, sales) VALUES (?, ?, ?)", sample_data)

# Commit changes and close the connection
conn.commit()
conn.close()

print("Database and table created successfully!")

Database and table created successfully!


In [2]:
from fpdf import FPDF

# Create a simple PDF
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.cell(200, 10, txt="Customer Feedback", ln=True, align="C")
pdf.ln(10)
pdf.multi_cell(0, 10, txt="Region A: Customers reported durability issues.\n"
                          "Region B: Customers are satisfied with the product.")
pdf.output("feedback.pdf")

print("Test PDF 'feedback.pdf' created successfully!")


Test PDF 'feedback.pdf' created successfully!


In [7]:
import os
from concurrent.futures import ThreadPoolExecutor
import sqlite3
import spacy
from PyPDF2 import PdfReader
from dotenv import load_dotenv
import requests

# Load environment variables
load_dotenv()

# The rest of the imports and classes remain the same until the SummarizationProcessor
# Only showing the modified parts for clarity
# === NLP-Based Query Parser === #
class QueryParser:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")  # Use SpaCy's local model

    def parse(self, query):
        doc = self.nlp(query)
        tasks = [f"Analyze {token.text.lower()}" for token in doc if token.dep_ in ("nsubj", "dobj")]
        return tasks

# === Extended Data Retrieval === #
class SQLDataSource:
    def __init__(self, db_path, query):
        self.db_path = db_path
        self.query = query

    def fetch_data(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute(self.query)
        data = cursor.fetchall()
        conn.close()
        return {"sql_data": data}


class DocumentParser:
    def __init__(self, file_path):
        self.file_path = file_path

    def fetch_data(self):
        if self.file_path.endswith(".pdf"):
            text = self._parse_pdf()
        else:
            raise ValueError("Unsupported file format.")
        return {"document_text": text}

    def _parse_pdf(self):
        reader = PdfReader(self.file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text


class APIDataSource:
    def __init__(self, api_url):
        self.api_url = api_url

    def fetch_data(self):
        # Mock API responses
        if "competitors" in self.api_url:
            return {"competitors": "Competitors in Region A offer durable products at competitive prices."}
        elif "market-trends" in self.api_url:
            return {"market_trends": "Growing demand for sustainable and durable products in Region A."}
        else:
            raise ValueError("Invalid API URL")


# === Parallel Execution === #
class ParallelReasoningPipeline:
    def __init__(self):
        self.steps = []

    def add_step(self, description, data_source, processor):
        self.steps.append({"description": description, "data_source": data_source, "processor": processor})

    def run(self):
        results = []

        def process_step(step):
            print(f"\n{step['description']}")
            data = step["data_source"].fetch_data()
            insights = step["processor"].process(data)
            print(f"Insights: {insights}")
            return insights

        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_step, step) for step in self.steps]
            for future in futures:
                results.append(future.result())

        return results


# === Enhanced Reasoning Logic === #
class SalesProcessor:
    def process(self, data):
        return f"Sales data: {data['sql_data']}"


class FeedbackProcessor:
    def process(self, data):
        return f"Customer feedback: {data['document_text'][:200]}..."  # Truncate for readability


class CompetitorProcessor:
    def process(self, data):
        return f"Competitor insights: {data['competitors']}"


class MarketTrendsProcessor:
    def process(self, data):
        return f"Market trends: {data['market_trends']}"

class SummarizationProcessor:
    """
    Summarizes insights using the Ollama API.
    """

    def __init__(self, model_name="tinyllama"):
        """
        Initialize the Ollama summarizer.
        :param model_name: Name of the Ollama model to use (default: tinyllama).
        """
        self.model_name = model_name
        self.api_url = "http://localhost:11434/api/generate"

    def process(self, insights):
        """
        Summarizes the combined insights using the Ollama API.
        :param insights: List of insights to summarize.
        :return: Summarized text.
        """
        combined_text = " ".join(insights)
        
        # Prepare the prompt
        prompt = f"""Please summarize the following insights concisely:

{combined_text}

Summary:"""

        # Prepare the request payload
        payload = {
            "model": self.model_name,
            "prompt": prompt,
            "stream": False
        }

        try:
            # Make request to Ollama API
            response = requests.post(self.api_url, json=payload)
            response.raise_for_status()
            
            # Extract the response
            result = response.json()
            return result.get('response', "Unable to generate summary.")
            
        except requests.exceptions.RequestException as e:
            return f"Error connecting to Ollama: {str(e)}"

def main():
    """
    Entry point for the program. Orchestrates query processing, reasoning steps, and summarization.
    """
    print(" Enhanced Multi-Hop Reasoning Agent \n")

    # Initialize components
    query = "What are the key factors driving the decline in sales for Product X in the last quarter?"
    query_parser = QueryParser()
    tasks = query_parser.parse(query)
    print(f"Parsed Tasks: {tasks}")

    # Initialize the pipeline
    pipeline = ParallelReasoningPipeline()

    # Add reasoning steps
    pipeline.add_step(
        "Analyze sales data",
        SQLDataSource("sales.db", "SELECT * FROM sales WHERE product='Product X'"),
        SalesProcessor(),
    )
    pipeline.add_step(
        "Analyze customer feedback",
        DocumentParser("feedback.pdf"),
        FeedbackProcessor(),
    )
    pipeline.add_step(
        "Analyze competitor data",
        APIDataSource("mock://competitors"),
        CompetitorProcessor(),
    )
    pipeline.add_step(
        "Analyze market trends",
        APIDataSource("mock://market-trends"),
        MarketTrendsProcessor(),
    )

    # Run the pipeline
    insights = pipeline.run()

    # Summarize results using Ollama
    summarizer = SummarizationProcessor(model_name="tinyllama")  # You can change the model as needed
    final_summary = summarizer.process(insights)

    print("\nFinal Summary")
    print(final_summary)

if __name__ == "__main__":
    main()

 Enhanced Multi-Hop Reasoning Agent 

Parsed Tasks: ['Analyze data']

Analyze sales data

Analyze customer feedback

Analyze competitor data
Insights: Competitor insights: Competitors in Region A offer durable products at competitive prices.

Analyze market trends
Insights: Market trends: Growing demand for sustainable and durable products in Region A.
Insights: Sales data: [(1, 'Product X', 'Region A', 100), (2, 'Product X', 'Region B', 200)]
Insights: Customer feedback: Customer Feedback
Region A: Customers reported durability issues.
Region B: Customers are satisfied with the product....

Final Summary
Salé, France - According to market research data, customer feedback reveals that in region A, consumers report issues with durability of the product. However, competitor insight suggests a trend towards growing demand for sustainable and durable products. The insight also highlights the changing demand patterns and increased consumer preferences for quality, reliability, and value-for

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import sqlite3
from PyPDF2 import PdfReader
from dotenv import load_dotenv
from rich.console import Console
from rich.panel import Panel
import requests

# Initialize console for Rich output
console = Console()

class QueryParser:
    """Parses user queries using LLM to extract actionable tasks"""
    
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def parse(self, query):
        """Extracts key analysis targets from user query"""
        prompt = f"""Analyze the following user query and identify the key factors or entities that need to be analyzed. 
        Return your response as a comma-separated list of terms, each term being a noun or noun phrase. 
        Do not include any additional text.

        Query: {query}
        Response:"""
        
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": prompt,
                "system": "You are a helpful assistant that extracts key analysis targets from user queries.",
                "stream": False
            }
        )
        
        response_text = response.json()["response"]
        tasks = [f"Analyze {term.strip()}" for term in response_text.split(",") if term.strip()]
        return tasks


# Data Sources #
class SQLDataSource:
    """Handles retrieving data from an SQLite database."""
    def __init__(self, db_path, query):
        self.db_path = db_path
        self.query = query

    def fetch_data(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute(self.query)
        data = cursor.fetchall()
        conn.close()
        return {"sql_data": data}


class DocumentParser:
    """Handles extracting text data from PDF documents."""
    def __init__(self, file_path):
        self.file_path = file_path

    def fetch_data(self):
        if self.file_path.endswith(".pdf"):
            text = self._parse_pdf()
        else:
            raise ValueError("Unsupported file format.")
        return {"document_text": text}

    def _parse_pdf(self):
        reader = PdfReader(self.file_path)
        text = "".join([page.extract_text() for page in reader.pages])
        return text


class APIDataSource:
    """Handles API data retrieval for competitor and market data."""
    def __init__(self, api_url):
        self.api_url = api_url

    def fetch_data(self):
        if "competitors" in self.api_url:
            return {"competitors": "Competitors in Region A offer durable products at competitive prices."}
        elif "market-trends" in self.api_url:
            return {"market_trends": "Growing demand for sustainable and durable products in Region A."}
        else:
            raise ValueError("Invalid API URL")


# Parallel Processing #
class ParallelReasoningPipeline:
    """Manages parallel execution of reasoning tasks with dynamic step numbering."""
    def __init__(self):
        self.steps = []

    def add_step(self, description, data_source, processor, data_type, color):
        """Adds an analysis step to the pipeline without predefined numbering."""
        self.steps.append({
            "description": description,
            "data_source": data_source,
            "processor": processor,
            "data_type": data_type,
            "color": color
        })

    def run(self):
        """Executes all pipeline steps in parallel and numbers them dynamically."""
        results = []
        step_number = 1  # Start numbering dynamically

        with ThreadPoolExecutor() as executor:
            future_to_step = {executor.submit(self.process_step, step): step for step in self.steps}

            for future in as_completed(future_to_step):
                step = future_to_step[future]
                insights = future.result()
                
                # Dynamically number the step
                step_content = (
                    f"[bold]Data Source:[/bold] {step['data_type']}\n"
                    f"[bold]Insights:[/bold] {insights}"
                )
                console.print(Panel(step_content, title=f"[bold]Step {step_number}: {step['description']}[/bold]", border_style=step["color"]))
                
                results.append(insights)
                step_number += 1  # Increment step number for next completed step

        return results

    def process_step(self, step):
        """Fetches data, processes it, and returns insights."""
        data = step["data_source"].fetch_data()
        return step["processor"].process(data)


# Data Processors #
class SalesProcessor:
    """Processes sales data using LLM for real insights."""
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def process(self, data):
        if not data["sql_data"]:
            return "No sales data available."

        # Format data for LLM analysis
        sales_records = "\n".join([str(record) for record in data["sql_data"]])

        # Prompt LLM to analyze sales trends
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": f"Analyze the following sales data and provide insights:\n\n{sales_records}",
                "system": "You are an expert business analyst.",
                "stream": False
            }
        )
        return response.json()["response"]


class FeedbackProcessor:
    """Processes customer feedback using LLM for real insights."""
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def process(self, data):
        feedback_text = data["document_text"]

        if not feedback_text.strip():
            return "No feedback data available."

        # Prompt LLM to analyze customer sentiment
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": f"Analyze the following customer feedback and provide insights:\n\n{feedback_text}",
                "system": "You are an expert in customer sentiment analysis.",
                "stream": False
            }
        )
        return response.json()["response"]


class CompetitorProcessor:
    """Processes competitor data using LLM for real insights."""
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def process(self, data):
        competitor_info = data.get("competitors", "")

        if not competitor_info.strip():
            return "No competitor data available."

        # Prompt LLM to analyze competitor strategy
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": f"Analyze the following competitor data and provide insights:\n\n{competitor_info}",
                "system": "You are an expert in competitor analysis.",
                "stream": False
            }
        )
        return response.json()["response"]


class MarketTrendsProcessor:
    """Processes market trends using LLM for real insights."""
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def process(self, data):
        market_trends_info = data.get("market_trends", "")

        if not market_trends_info.strip():
            return "No market trend data available."

        # Prompt LLM to analyze market trends
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": f"Analyze the following market trend data and provide insights:\n\n{market_trends_info}",
                "system": "You are an expert market analyst.",
                "stream": False
            }
        )
        return response.json()["response"]


class SummarizationProcessor:
    """Summarizes insights using LLM."""
    def __init__(self):
        self.base_url = "http://localhost:11434/api/generate"

    def process(self, insights):
        """Generates final summary using LLM."""
        combined_text = " ".join(insights)
        
        response = requests.post(
            self.base_url,
            json={
                "model": "tinyllama",
                "prompt": combined_text,
                "system": "Create a concise, actionable summary of the following business insights.",
                "stream": False
            }
        )
        return response.json()["response"]


def main():
    console.print("[bold cyan]\nEnhanced Multi-Hop Reasoning Agent[/bold cyan]\n")

    # Parse the query using LLM
    query = "What are the key factors driving the decline in sales for Product X in the last quarter?"
    query_parser = QueryParser()
    tasks = query_parser.parse(query)
    console.print(Panel(f"Parsed Tasks: {tasks}", title="Query Parsing", border_style="cyan"))

    # Initialize the pipeline
    pipeline = ParallelReasoningPipeline()

    # Add steps to pipeline
    pipeline.add_step(
        "Retrieve and analyze sales data for Product X",
        SQLDataSource("sales.db", "SELECT * FROM sales WHERE product='Product X'"),
        SalesProcessor(),
        "SQL Database", "blue"
    )
    pipeline.add_step(
        "Retrieve and analyze customer feedback for Product X",
        DocumentParser("feedback.pdf"),
        FeedbackProcessor(),
        "Document", "green"
    )
    pipeline.add_step(
        "Retrieve and analyze competitor data",
        APIDataSource("mock://competitors"),
        CompetitorProcessor(),
        "API", "yellow"
    )
    pipeline.add_step(
        "Retrieve and analyze market trends",
        APIDataSource("mock://market-trends"),
        MarketTrendsProcessor(),
        "API", "red"
    )

    # Run the pipeline
    insights = pipeline.run()

    # Summarize results using LLM
    summarizer = SummarizationProcessor()
    final_summary = summarizer.process(insights)

    console.print(Panel(final_summary, title="[bold]Final Summary[/bold]", border_style="magenta"))


if __name__ == "__main__":
    main()

Enhanced Multi-Hop Reasoning Agent

╭───────────────────────────────────────────────── Query Parsing ─────────────────────────────────────────────────╮
│ Parsed Tasks: ['Analyze Key Factors Driving Decline in Sales for Product X in the Last Quarter:\n1. Increased   │
│ competition among competitors in the market\n2. Limited availability of products due to overstocking by         │
│ competitors\n3. Poor customer service or delivery issues\n4. Lack of innovation in product offerings\n5.        │
│ Changes in consumer buying habits due to COVID-19 pandemic', 'Analyze such as shift towards online shopping and │
│ changing preferences for eco-friendly products']                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────── Step 1: Retrieve and analyze customer feedback for Product X ──────────────────────────╮
│ Data Source: Document                                                                                           │
│ Insights: Regarding region A, customers reported issues with the durability of their products, which were seen  │
│ as a major concern for them. The feedback was generally negative, indicating that these customers had           │
│ experienced issues with the quality or performance of the product, regardless of how they interacted with it.   │
│ This could indicate that some of these products may have been manufactured with subpar materials or quality     │
│ control measures, which caused issues during use or handling.                                                   │
│                                                                                                                 │
│ Regarding region B, customers were generally satisfied with their products and had reported positive            │
│ experiences with their purchases. The feedback indicated that the products met or exceeded expectations for     │
│ quality, functionality, and overall satisfaction. It's clear from the customer feedback that the products are   │
│ meeting their needs and providing value for money.                                                              │
│                                                                                                                 │
│ Overall, the analysis revealed that there were some issues with durability in region A but not enough to        │
│ significantly affect overall customer satisfaction. This suggests that manufacturers should continue to ensure  │
│ that product quality is top-notch to avoid any negative feedback from customers. The positive feedback from     │
│ customers in region B indicates that the products are of high quality and provide satisfactory experiences,     │
│ which shows that there's room for improvement.                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────── Step 2: Retrieve and analyze sales data for Product X ─────────────────────────────╮
│ Data Source: SQL Database                                                                                       │
│ Insights: I don't have access to the specific sales data provided in your text. However, based on the           │
│ description of product x and regions a and b, here are some insights that you can provide:                      │
│                                                                                                                 │
│ 1. Product x sold well in region a, while it performed poorly in region b. This may indicate that the marketing │
│ strategy for region a was more effective than that for region b. Regions with higher sales volumes typically    │
│ receive more promotional support from vendors, which can lead to better overall results.                        │
│                                                                                                                 │
│ 2. The high sales volume for product x in region a may be attributed to its unique value proposition or         │
│ superior quality compared to similar products sold in region b. This could make it more appealing to customers  │
│ and increase customer loyalty to the vendor.                                                                    │
│                                                                                                                 │
│ 3. By analyzing the data by regions, you can identify which vendors are performing better in each region and    │
│ target your marketing efforts accordingly. This approach enables you to maximize revenue and achieve greater    │
│ profitability.                                                                                                  │
│                                                                                                                 │
│ 4. You should also consider the competition in the respective regions, as well as their market shares. By       │
│ understanding the strengths and weaknesses of the competitors, you can develop targeted campaigns that aim to   │
│ differentiate yourself from them and improve your overall performance.                                          │
│                                                                                                                 │
│ 5. Finally, you may want to analyze sales data by product category to gain insight into which products are      │
│ performing best in each region. This could enable you to optimize your marketing efforts based on the most      │
│ popular products or categories.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────── Step 3: Retrieve and analyze competitor data ──────────────────────────────────╮
│ Data Source: API                                                                                                │
│ Insights: 1. Company A: Company A offers a wide range of durable products at competitive prices, which is one   │
│ of its main selling points. The company uses quality materials and implements advanced manufacturing techniques │
│ to ensure products are durable, long-lasting, and provide value for customers.                                  │
│                                                                                                                 │
│ Competitors in Region B offer similar products but with lower prices due to economies of scale or pricing       │
│ strategies aimed at reducing costs. However, Company A offers superior quality and customer service, making it  │
│ a popular choice among consumers who prioritize durability and reliability.                                     │
│                                                                                                                 │
│ 2. Company B: This competitor offers a wide range of products at lower prices due to economies of scale or      │
│ pricing strategies aimed at reducing costs. However, Company B offers inferior quality products that are not as │
│ durable as Company A's products, which leads to less customer loyalty and a decline in sales.                   │
│                                                                                                                 │
│ 3. Company C: This competitor is known for offering high-quality products at higher prices due to the company's │
│ reputation for producing high-end products that offer premium features and superior quality. However, the       │
│ company's pricing strategy may be too expensive for some consumers who prioritize affordability over other      │
│ factors like durability or reliability.                                                                         │
│                                                                                                                 │
│ In summary, Company A offers a wide range of durable products at competitive prices, while Company B offers     │
│ lower-quality products at lower prices due to economies of scale and pricing strategies aimed at reducing       │
│ costs. However, Company C offers high-quality products that offer premium features but at higher prices. The    │
│ company's pricing strategy may not appeal to some consumers who prioritize affordability over other factors     │
│ like durability or reliability.                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────── Step 4: Retrieve and analyze market trends ───────────────────────────────────╮
│ Data Source: API                                                                                                │
│ Insights: The "Market Trends for Sustainable and Durable Products" report by Grand View Research Inc. Analyzes  │
│ the Growi ng demand for sustainable and durable products in Region A. The report covers the following regions:  │
│ North America, Europe, Asia Pacific, and the Rest of the World (RoW). It provides a detailed analysis of the    │
│ factors driving the growth of the market and key trends and opportunities.                                      │
│                                                                                                                 │
│ Key Trends:                                                                                                     │
│                                                                                                                 │
│ 1. Increasing consumer awareness about sustainability and eco-friendly products                                 │
│ 2. Increasing government initiatives for promoting sustainable practices and encouraging the use of             │
│ environmentally friendly materials                                                                              │
│ 3. Consumers preferring sustainable and durable products over disposable ones                                   │
│ 4. The growing popularity of eco-conscious lifestyles among consumers, particularly in developing countries     │
│                                                                                                                 │
│ Opportunities:                                                                                                  │
│                                                                                                                 │
│ 1. Increasing demand for natural ingredients and plant-based materials in the manufacturing process of          │
│ sustainable and durable products                                                                                │
│ 2. The rising demand for renewable energy sources, which will create a need for electricity and heating systems │
│ that are made from sustainable and durable materials                                                            │
│ 3. Growing investments in research and development to improve the durability and functionality of sustainable   │
│ and durable products                                                                                            │
│                                                                                                                 │
│ Market Size:                                                                                                    │
│                                                                                                                 │
│ The global market size for sustainable and durable products is expected to reach $1.79 trillion by 2028, with   │
│ North America accounting for a significant share (46%). The Asia Pacific region will have the highest growth    │
│ rate in this market during the forecast period.                                                                 │
│                                                                                                                 │
│ Impact of Covid-19 Pandemic:                                                                                    │
│                                                                                                                 │
│ The pandemic has had a profound impact on sustainable and durable product demand. The use of eco-friendly       │
│ materials and green packaging has increased, resulting in a decline in overall demand for traditional products. │
│ However, the pandemic is expected to drive a surge in 